In [1]:
import random
import time
import os

import cv2
import numpy as np

import torch
import torchvision
import torch.nn as nn

from PIL import Image
from IPython import display

from torchvision.transforms import transforms
from torchsummary import summary

Ref: Section 7 (Appendix) from CycleGAN

## Generator Architecture

In [20]:
# CINR = Convolution Instance Norm ReLU
# 3x3 convolutions with stride 1/2, 1 or 2 depending on position
# Uses reflection padding
# In the paper:
# dk denotes a k filter stride 2 with 3x3 conv
# c7s1-k denotes a k filter stride 1 with 7x7 conv
# uk denotes a k filter stride 1/2 with 3x3 conv
class CINRLayer(nn.Module):
    def __init__(self, in_ch, out_ch, stride, kernel_size, reflect_pad):
        super().__init__()
        
        layers = []
        
        conv_type = nn.Conv2d if stride >= 1 else nn.ConvTranspose2d
        stride = stride if stride >= 1 else int(1 / stride)
        
        if reflect_pad:
            layers.append(nn.ReflectionPad2d(kernel_size // 2))
        
        layers += [
            conv_type(in_ch, out_ch, kernel_size=kernel_size, stride=stride, bias=True),
            nn.InstanceNorm2d(out_ch),
            nn.ReLU(True)
        ]
        
        self.seq = nn.Sequential(*layers)
    
    def forward(self, batch):
        return self.seq(batch)

In [21]:
# Contains 2 3x3 convolutional layers with the same number of filters on both layers
# Use reflect padding in these
# Don't use dropout
# Use instancenorm
class GeneratorResidualBlock(nn.Module):
    def __init__(self, feature_size):
        super().__init__()
        
        layers = [
            nn.ReflectionPad2d(1),
            nn.Conv2d(feature_size, feature_size, kernel_size=3, bias=True),
            nn.InstanceNorm2d(feature_size),
            nn.ReLU(True),
            # Dropout would go here if I want it
            nn.ReflectionPad2d(1),
            nn.Conv2d(feature_size, feature_size, kernel_size=3, bias=True)
        ]
        
        self.seq = nn.Sequential(*layers)
    
    def forward(self, batch):
        return batch + self.seq(batch)

In [22]:
# For the 128x128 case:
# c7s1-64, d128, d256, R256 x 6, u128, u64, c7s1-3
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        
        layers = [
            CINRLayer(in_ch=3, out_ch=64, stride=1, kernel_size=7, reflect_pad=True),
            CINRLayer(in_ch=64, out_ch=128, stride=2, kernel_size=3, reflect_pad=False),
            CINRLayer(in_ch=128, out_ch=256, stride=2, kernel_size=3, reflect_pad=False),
            
            # same dim all the way through
            GeneratorResidualBlock(feature_size=256),
            GeneratorResidualBlock(feature_size=256),
            GeneratorResidualBlock(feature_size=256),
            GeneratorResidualBlock(feature_size=256),
            GeneratorResidualBlock(feature_size=256),
            GeneratorResidualBlock(feature_size=256),
            
            CINRLayer(in_ch=256, out_ch=128, kernel_size=3, stride=0.5, reflect_pad=False),
            CINRLayer(in_ch=128, out_ch=64, kernel_size=3, stride=0.5, reflect_pad=False),
            CINRLayer(in_ch=64, out_ch=3, stride=1, kernel_size=7, reflect_pad=True)
        ]
        
        self.seq = nn.Sequential(*layers)
        
    def forward(self, batch):
        return self.seq(batch)

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
if (torch.cuda.device_count()>0):
    print(torch.cuda.get_device_name(0))
    
generator_model = Generator().to(device)
generator_model

cuda
NVIDIA GeForce GTX 1080


Generator(
  (seq): Sequential(
    (0): CINRLayer(
      (seq): Sequential(
        (0): ReflectionPad2d((3, 3, 3, 3))
        (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
        (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (3): ReLU(inplace=True)
      )
    )
    (1): CINRLayer(
      (seq): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
        (1): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): ReLU(inplace=True)
      )
    )
    (2): CINRLayer(
      (seq): Sequential(
        (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2))
        (1): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (2): ReLU(inplace=True)
      )
    )
    (3): GeneratorResidualBlock(
      (seq): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))

In [24]:
summary(generator_model, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ReflectionPad2d-1          [-1, 3, 134, 134]               0
            Conv2d-2         [-1, 64, 128, 128]           9,472
    InstanceNorm2d-3         [-1, 64, 128, 128]               0
              ReLU-4         [-1, 64, 128, 128]               0
         CINRLayer-5         [-1, 64, 128, 128]               0
            Conv2d-6          [-1, 128, 63, 63]          73,856
    InstanceNorm2d-7          [-1, 128, 63, 63]               0
              ReLU-8          [-1, 128, 63, 63]               0
         CINRLayer-9          [-1, 128, 63, 63]               0
           Conv2d-10          [-1, 256, 31, 31]         295,168
   InstanceNorm2d-11          [-1, 256, 31, 31]               0
             ReLU-12          [-1, 256, 31, 31]               0
        CINRLayer-13          [-1, 256, 31, 31]               0
  ReflectionPad2d-14          [-1, 256,

Todo: have a small shrink here